### Example: Analyzing Citibike Station Activity using Pandas

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])

We are going to use the database of snapshots of Citibike stations statuses. 

We will first fetch the data from the database.

In [ ]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')
engine = create_engine(conn_string)

In [ ]:
query = '''
SELECT * 
FROM citibike.stations
'''
df = pd.read_sql(query, con=engine)

In [ ]:
len(df)

In [ ]:
df.head(10)

In [ ]:
df.dtypes

### Exploratory Analysis

As a first step, let's see how the status of the bike stations evolves over time. We compute the average "fullness" of all the bike stations over time. We can use the `groupby` function of pandas, and compute the `mean()` for the groups.

In [ ]:
# Notice that this also returns an average for the station ID's which is kind of useless
# We will eliminate these next.
df.groupby('communication_time').mean()

Now let's plot the activity over time. We can see that the percentage of bikes in the stations falls from 35% overnight to 30% during the morning and evening commute times, while the average availability during the day is around 31%.

In [ ]:
df.groupby('communication_time').mean().percent_full.plot(
    figsize=(20,10), grid=True
)

Let's do also the seasonal decomposition to see the result.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

time_series = df.groupby('communication_time').mean().percent_full

# We decompose assumming a 24-hour periodicity. 
# There is a weekly component as well, which we ignore.
# We can also specify a multiplicative instead of an additive model
# The additive model is Y[t] = T[t] + S[t] + e[t]
# The multiplicative model is Y[t] = T[t] * S[t] * e[t]
decomposition = seasonal_decompose(time_series, model='multiplicative', freq=24)

seasonal = decomposition.plot()  
seasonal.set_size_inches(20, 10)

### Examining Time Series per Station

We now create a pivot table, to examine the time series for individual stations.

Notice that we use the `fillna` method, where we fill the cells where we do not have values using the prior, non-missing value.

In [ ]:
import numpy as np
df2 = df.pivot_table(
    index='communication_time', 
    columns='id', 
    values='percent_full', 
    aggfunc=np.mean
).interpolate(method='time') 
df2

Let's plot the time series for *all* bike stations, for a couple of days in February.

In [ ]:
df2.plot(
    alpha=0.05, 
    color='b', 
    legend=False, 
    figsize=(20,10), 
    xlim=('2017-02-15','2017-02-19')
)

Let's limit our plot to just two stations:
* Station 3260 at "Mercer St & Bleecker St"
* Station 161 at "LaGuardia Pl & W 3 St"

which are nearby and tend to exhibit similar behavior. Remember that the list of stations is [available as a JSON](https://feeds.citibikenyc.com/stations/stations.json) 

In [ ]:
df2[[161, 3260]].plot(
    alpha=0.5,  
    legend=False, 
    figsize=(20,10), 
   xlim=('2017-02-15','2017-02-27')
)

### Finding Bike Stations with Similar Behavior

For our next analysis, we are going to try to find bike stations that have similar behaviors over time. A very simple technique that we can use to find similar time series is to treat the time series as vectors, and compute their correlation. Pandas provides the `corr` function that can be used to calculate the correlation of columns. (If we want to compute the correlation of rows, we can just take the transpose of the dataframe using the `transpose()` function, and compute the correlations there.)

In [ ]:
similarities = df2.corr(method='pearson')
similarities

Let's see the similarities of the two stations that we examined above.

In [ ]:
stations = [161, 3260]

similarities[stations].loc[stations]

For bookkeeping purposes, we are going to drop rows that contain NaN values, as we cannot use such similarity values.

In [ ]:
similarities.dropna(axis='index', how='any', inplace=True)

We are now going to convert our similarities into distance metrics, that are positive, and bounded to be between 0 and 1.

* If two stations have correlation 1, they behave identically, and therefore have distance 0, 
* If two stations have correlation -1, they have exactly the oppositite behaviors, and therefore we want to have distance 1 (the max) 

In [ ]:
distances = ((.5*(1-similarities))**2)
distances

### Clustering Based on Distances

Without explaining too much about clustering, we are going to use a clustering technique and cluster together bike stations that are "nearby" according to our similarity analysis. The code is very simple:

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=2)
cluster.fit(distances.values)

We will now take the results of the clustering and associate each of the data points into a cluster.

In [ ]:
labels = pd.DataFrame(list(zip(distances.index.values.tolist(), cluster.labels_)), columns = ["id", "cluster"])
labels.head(10)

Let's see how many stations in each cluster

In [ ]:
labels.groupby('cluster').count()

### Visualizing the Time Series Clusters

We will start by assining a color to each cluster, so that we can plot each station-timeline with the cluster color. (We put a long list of colors, so that we can play with the number of clusters in the earlier code, and still get nicely colored results.)

In [ ]:
colors = list(['r','b', 'g', 'm', 'y', 'k', 'w', 'c'])
labels['color'] = labels['cluster'].apply(lambda cluster_id : colors[cluster_id]) 
labels.head(10)

In [ ]:
stations_plot = df2.plot(
    alpha=0.02, 
    legend=False, 
    figsize=(20,5), 
    color=labels["color"],
    #xlim=('2017-02-15','2017-02-17')
)

The plot still looks messy. Let's try to plot instead a single line for each cluster. To represent the cluster, we are going to use the _median_ fullness value across all stations that belong to a cluster, for each timestamp. For that, we can again use a pivot table: we define the `timestamp` as one dimension of the table, and `cluster` as the other dimension, and we use the `percentile` function to compute the median. 

For that, we first _join_ our original dataframe with the results of the clustering, using the `merge` command, and add an extra column that includes the clusterid for each station. Then we compute the pivot table.

In [ ]:
import numpy as np

median_cluster = df.merge(
    labels, 
    how='inner', 
    on='id'
).pivot_table(
    index='communication_time', 
    columns='cluster', 
    values='percent_full', 
    aggfunc=lambda x: np.percentile(x, 50) # median
)

median_cluster

Now, we can plot the medians for the two clusters.

In [ ]:
median_plot = median_cluster.plot(
        figsize=(20,5), 
        linewidth = 2, 
        alpha = 0.75,
        color=colors,
        ylim = (0,0.75),
        xlim=('2017-02-13','2017-02-20'),
        grid = True
    )

And just for fun and for visual decoration, let's put the two plots together. We are going to fade a lot the individual station time series (by putting the `alpha=0.01`) and we are going to make more prominent the median lines by increasing their linewidths. We will limit our plot to one week's worth of data:

In [ ]:
stations_plot = df2.plot(
    alpha=0.01, 
    legend=False, 
    figsize=(20,5), 
    color=labels["color"]
)

median_cluster.plot(
    figsize=(20,5), 
    linewidth = 3, 
    alpha = 0.5,
    color=colors, 
    xlim=('2017-02-13','2017-02-20'),
    ax = stations_plot
)